In [89]:
#importing the libraries
import tensorflow as tf
import numpy as np
import random
import string
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
#importing the dataset
with open('content.json') as content:
    data1 = json.load(content)

In [99]:
def get_train_test_data(data1):
    tags = []
    inputs = []
    responses={}
    for intent in data1['intents']:
      responses[intent['tag']]=intent['responses']
      for lines in intent['input']:
        inputs.append(lines)
        tags.append(intent['tag'])
    #converting to dataframe
    data = pd.DataFrame({"inputs":inputs,
                         "tags":tags})
    # print(data)
    #removing punctuations
    data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
    data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
    #tokenize the data
    
#     def get_data(data):
    tokenizer = Tokenizer(num_words=2000)
    tokenizer.fit_on_texts(data['inputs'])
    train = tokenizer.texts_to_sequences(data['inputs'])

    #apply padding

    X = pad_sequences(train)

    #encoding the outputs

    le = LabelEncoder()
    y = le.fit_transform(data['tags'])

        
#         return x_train,y_train
        
    n = int(len(X)*0.8)
    x_train,y_train = X[:n],y[:n]
    x_test,y_test = X[n:],y[n:]
    
    return x_train,y_train,x_test,y_test

In [100]:
x_train,y_train,x_test,y_test = get_train_test_data(data1)

In [60]:
def get_trained_model(data1):
    #getting all the data to lists
#     tags = []
#     inputs = []
#     responses={}
#     for intent in data1['intents']:
#       responses[intent['tag']]=intent['responses']
#       for lines in intent['input']:
#         inputs.append(lines)
#         tags.append(intent['tag'])
#     #converting to dataframe
#     data = pd.DataFrame({"inputs":inputs,
#                          "tags":tags})
#     # print(data)
#     #removing punctuations
#     data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
#     data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#     #tokenize the data
    
#     tokenizer = Tokenizer(num_words=2000)
#     tokenizer.fit_on_texts(data['inputs'])
#     train = tokenizer.texts_to_sequences(data['inputs'])

#     #apply padding
    
#     x_train = pad_sequences(train)

#     #encoding the outputs
    
#     le = LabelEncoder()
#     y_train = le.fit_transform(data['tags'])
    #input length
    input_shape = x_train.shape[1]
    # print(input_shape)
    #define vocabulary
    vocabulary = len(tokenizer.word_index)
    print("number of unique words : ",vocabulary)
    #output length
    output_length = le.classes_.shape[0]
    print("output length: ",output_length)
    #creating the model
    i = Input(shape=(input_shape,))
    x = Embedding(vocabulary+1,10)(i)
    x = LSTM(10,return_sequences=True)(x)
    x = Flatten()(x)
    x = Dense(output_length,activation="softmax")(x)
    model  = Model(i,x)
    #compiling the model
    model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
    #training the model
    score = model.fit(x_train,y_train,epochs=200,verbose=1)
    
    print('Accuracy:',score)
    return responses,le,input_shape,tokenizer,model,x_train,y_train

In [61]:
#chatting
def start_chat(responses,le,input_shape,tokenizer,model):
    while True:
        texts_p = []
        prediction_input = input('You : ')
        #removing punctuation and converting to lowercase
        prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
        prediction_input = ''.join(prediction_input)
        texts_p.append(prediction_input)
        #tokenizing and padding
        prediction_input = tokenizer.texts_to_sequences(texts_p)
        prediction_input = np.array(prediction_input).reshape(-1)
        prediction_input = pad_sequences([prediction_input],input_shape)
        #getting output from model
        output = model.predict(prediction_input)
        output = output.argmax()
        #finding the right tag and predicting
        response_tag = le.inverse_transform([output])[0]
        print("Going Merry : ",random.choice(responses[response_tag]))
        if response_tag == "goodbye":
            break

# Dataset1 : content.json

In [62]:
responses1,le1, input_shape1, tokenizer1, model1,x_train,y_train = get_trained_model(data1)

number of unique words :  96
output length:  8
Epoch 1/200
3/3 [==============================] - 2s 4ms/step - loss: 2.0765 - accuracy: 0.1975
Epoch 2/200
3/3 [==============================] - 0s 4ms/step - loss: 2.0724 - accuracy: 0.2099
Epoch 3/200
3/3 [==============================] - 0s 4ms/step - loss: 2.0683 - accuracy: 0.1975
Epoch 4/200
3/3 [==============================] - 0s 4ms/step - loss: 2.0636 - accuracy: 0.1975
Epoch 5/200
3/3 [==============================] - 0s 3ms/step - loss: 2.0594 - accuracy: 0.1975
Epoch 6/200
3/3 [==============================] - 0s 4ms/step - loss: 2.0539 - accuracy: 0.1975
Epoch 7/200
3/3 [==============================] - 0s 4ms/step - loss: 2.0486 - accuracy: 0.1975
Epoch 8/200
3/3 [==============================] - 0s 4ms/step - loss: 2.0425 - accuracy: 0.1975
Epoch 9/200
3/3 [==============================] - 0s 4ms/step - loss: 2.0358 - accuracy: 0.1975
Epoch 10/200
3/3 [==============================] - 0s 5ms/step - loss: 2.0284 -

3/3 [==============================] - 0s 4ms/step - loss: 0.1892 - accuracy: 0.9877
Epoch 168/200
3/3 [==============================] - 0s 4ms/step - loss: 0.1849 - accuracy: 0.9877
Epoch 169/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1819 - accuracy: 0.9877
Epoch 170/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1781 - accuracy: 0.9877
Epoch 171/200
3/3 [==============================] - 0s 4ms/step - loss: 0.1780 - accuracy: 0.9877
Epoch 172/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1742 - accuracy: 0.9877
Epoch 173/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1695 - accuracy: 0.9877
Epoch 174/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1690 - accuracy: 0.9877
Epoch 175/200
3/3 [==============================] - 0s 4ms/step - loss: 0.1671 - accuracy: 0.9877
Epoch 176/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1616 - accuracy: 0.9877
Epoch 177/200
3/3 [=====

In [40]:
start_chat(responses1,le1,input_shape1,tokenizer1,model1)

You : hi
1/1 [==============================] - 0s 22ms/step
Going Merry :  Hi random person, what do'ya want ?
You : hi
1/1 [==============================] - 0s 19ms/step
Going Merry :  Ha , Glad you showed up. How are you ?
You : hi
1/1 [==============================] - 0s 17ms/step
Going Merry :  Hello my friend, How can I help you ?
You : this
1/1 [==============================] - 0s 19ms/step
Going Merry :  Okay , Take care


# New Dataset

In [41]:
dfs = pd.read_excel(r"D:\OneDrive - NITT\Custom_Download\final data.csv", sheet_name=None)

df1 = dfs['Sheet1']
df1 = df1.dropna()

df = df1.groupby('tag').agg({"pattern": lambda x:list(x),
                       "response" : lambda x: list(x)
                       })
df = df.reset_index(level=0)
df = df.rename(columns={'pattern':'input','response':'responses'})
corpus = df.to_dict("records")
data2 = {}
data2['intents'] = corpus

In [42]:
responses2,le2, input_shape2, tokenizer2, model2 = get_trained_model(data1)

number of unique words :  96
output length:  8
Epoch 1/200
3/3 [==============================] - 2s 5ms/step - loss: 2.0789 - accuracy: 0.0494
Epoch 2/200
3/3 [==============================] - 0s 5ms/step - loss: 2.0752 - accuracy: 0.1481
Epoch 3/200
3/3 [==============================] - 0s 6ms/step - loss: 2.0717 - accuracy: 0.1852
Epoch 4/200
3/3 [==============================] - 0s 5ms/step - loss: 2.0686 - accuracy: 0.1975
Epoch 5/200
3/3 [==============================] - 0s 5ms/step - loss: 2.0652 - accuracy: 0.1852
Epoch 6/200
3/3 [==============================] - 0s 5ms/step - loss: 2.0612 - accuracy: 0.1852
Epoch 7/200
3/3 [==============================] - 0s 7ms/step - loss: 2.0575 - accuracy: 0.1852
Epoch 8/200
3/3 [==============================] - 0s 5ms/step - loss: 2.0527 - accuracy: 0.1975
Epoch 9/200
3/3 [==============================] - 0s 5ms/step - loss: 2.0481 - accuracy: 0.2222
Epoch 10/200
3/3 [==============================] - 0s 6ms/step - loss: 2.0428 -

3/3 [==============================] - 0s 4ms/step - loss: 0.1111 - accuracy: 0.9877
Epoch 168/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1098 - accuracy: 0.9877
Epoch 169/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1081 - accuracy: 0.9877
Epoch 170/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1064 - accuracy: 0.9877
Epoch 171/200
3/3 [==============================] - 0s 6ms/step - loss: 0.1048 - accuracy: 0.9877
Epoch 172/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1046 - accuracy: 0.9877
Epoch 173/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1022 - accuracy: 0.9877
Epoch 174/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1005 - accuracy: 0.9877
Epoch 175/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0988 - accuracy: 0.9877
Epoch 176/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0984 - accuracy: 0.9877
Epoch 177/200
3/3 [=====

In [53]:
start_chat(responses2,le2, input_shape2, tokenizer2, model2)

You : hi
1/1 [==============================] - 0s 18ms/step
Going Merry :  Ha , Glad you showed up. How are you ?
You : how are you
1/1 [==============================] - 0s 17ms/step
Going Merry :  Things are Great
You : are u ok
1/1 [==============================] - 0s 19ms/step
Going Merry :  welcome aboard, how may I help you ?
You : r==are you happy
1/1 [==============================] - 0s 24ms/step
Going Merry :  Ha , Glad you showed up. How are you ?
You : fine
1/1 [==============================] - 0s 17ms/step
Going Merry :  Goodbye


In [ ]:
https://www.kaggle.com/datasets/elvinagammed/chatbots-intent-recognition-dataset

In [65]:
score = model1.evaluate(x_train, y_train)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

3/3 [==============================] - 3s 4ms/step - loss: 0.1131 - accuracy: 1.0000

ACCURACY: 1.0
LOSS: 0.11306800693273544


In [64]:
model1